# IMPORT LIBRARY

In [288]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [289]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [290]:
# Find the path of the stopwords resource
stopwords_path = nltk.data.find('corpora/stopwords.zip')
print(f"The stopwords resource is located at: {stopwords_path}")

The stopwords resource is located at: /root/nltk_data/corpora/stopwords.zip


# LOADING DATA

In [291]:
import os
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tugas Akhir Praktisi Text Mining/dataset_sms_spam_v1.csv')
print(data.shape)
data.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(1143, 2)


,Teks,label
0,[PROMO] Beli paket Flash mulai 1GB di MY TELKO...,2
1,2.5 GB/30 hari hanya Rp 35 Ribu Spesial buat A...,2
2,"2016-07-08 11:47:11.Plg Yth, sisa kuota Flash ...",2
3,"2016-08-07 11:29:47.Plg Yth, sisa kuota Flash ...",2
4,4.5GB/30 hari hanya Rp 55 Ribu Spesial buat an...,2


# EDA

In [292]:
# CLASS DISTRIBUTION
#if dataset is balanced or not
x=data['label'].value_counts()
x

,count
label,
0,569
1,335
2,239


In [293]:
#Missing values
data.isna().sum()

,0
Teks,0
label,0


# TEXT PREPROCESSING

## CASE FOLDING(TEXT CLEANING)

In [294]:
import re

# Membuat fungsi untuk case folding
def casefolding(text):
    text = text.lower()                               # membuat kalimat menjadi huruf kecil
    text = re.sub(r'https?://\S+|www\.\S+', '', text) # menghapus url dari kalimat
    text = re.sub(r'[-+]?[0-9]+', '', text)           # menghapus angka dari kalimat
    text = re.sub(r'[^\w\s]', '', text)               # menghapus tanda baca dari kalimat
    text = re.sub(r'\s+', ' ', text)                  # menghapus spasi ganda dari kalimat
    text = text.strip()                               # menghapus spasi di awal dan akhir kalimat

    return text

In [295]:
data_sample = data['Teks'].iloc[3]
case_folding = casefolding(data_sample)
print('Test Sample\t : ', data_sample)
print('Case Folding\t : ', case_folding)

Test Sample	 :  2016-08-07 11:29:47.Plg Yth, sisa kuota Flash Anda 7160KB. Download MyTelkomsel apps di http://tsel.me/tsel utk cek kuota&beli paket Flash atau hub *363#
Case Folding	 :  plg yth sisa kuota flash anda kb download mytelkomsel apps di utk cek kuotabeli paket flash atau hub


## WORD NORMALIZATION

In [296]:
key_norm = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tugas Akhir Praktisi Text Mining/key_norm.csv')
key_norm.head()

,_id,singkat,hasil
0,1,abis,habis
1,2,accent,tekanan
2,3,accept,terima
3,4,accident,kecelakaan
4,5,achievement,prestasi


In [297]:
def textnormalize(text):
    text = ' '.join([key_norm[key_norm['singkat'] == word]['hasil'].values[0]
    if (key_norm['singkat'] == word).any()
    else word for word in text.split()])

    text = str.lower(text)
    return text

In [298]:
data_sample = data['Teks'].iloc[696]
word_normal = textnormalize(data_sample)
print('Data Sample\t : ', data_sample)
print('Word Normalizing\t : ', word_normal)


Data Sample	 :  Btw magicomnya yg sedang Gais, gaada yg gede
Word Normalizing	 :  ngomong ngomong magicomnya yang sedang gais, tidak ada yang besar


## STOPWORD REMOVAL

In [299]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stopwords_indonesia = stopwords.words('indonesian')

In [300]:
len(stopwords_indonesia)

758

In [301]:
stopwords_indonesia

['ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir',
 'akhiri',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'anda',
 'andalah',
 'antar',
 'antara',
 'antaranya',
 'apa',
 'apaan',
 'apabila',
 'apakah',
 'apalagi',
 'apatah',
 'artinya',
 'asal',
 'asalkan',
 'atas',
 'atau',
 'ataukah',
 'ataupun',
 'awal',
 'awalnya',
 'bagai',
 'bagaikan',
 'bagaimana',
 'bagaimanakah',
 'bagaimanapun',
 'bagi',
 'bagian',
 'bahkan',
 'bahwa',
 'bahwasanya',
 'baik',
 'bakal',
 'bakalan',
 'balik',
 'banyak',
 'bapak',
 'baru',
 'bawah',
 'beberapa',
 'begini',
 'beginian',
 'beginikah',
 'beginilah',
 'begitu',
 'begitukah',
 'begitulah',
 'begitupun',
 'bekerja',
 'belakang',
 'belakangan',
 'belum',
 'belumlah',
 'benar',
 'benarkah',
 'benarlah',
 'berada',
 'berakhir',
 'berakhirlah',
 'berakhirnya',
 'berapa',
 'berapakah',
 'berapalah',
 'berapapun',
 'berarti',
 'berawal',
 'berbagai',
 'berdatangan',
 'beri',
 'berikan',
 'berikut'

In [302]:
# menambahkan kata dalam stopword
more_stopword = ['tsel', 'telkomsel', 'xl', 'rb', 'btw']
stopwords_indonesia = stopwords_indonesia + more_stopword

# mebuat fungsi stopword removal
def stopwordremoval(text):
    clean_words = []
    text = text.split()
    for word in text:
        if word not in stopwords_indonesia:
            clean_words.append(word)
    return ' '.join(clean_words)

In [303]:
data_sample = data['Teks'].iloc[500]
case_folding = casefolding(data_sample)
word_norm = textnormalize(case_folding)
stopword_removal = stopwordremoval(word_norm)

print('Test Sample\t\t : ', data_sample)
print('Case Folding\t\t : ', case_folding)
print('Word Nomalisasi\t\t : ', word_norm)
print('Stopword Removal\t : ', stopword_removal)

Test Sample		 :  Slmt No Anda mendptkan Hadiah 1 unit MOBIL HONDA jazz dr promo TRI CARE 2016 dgn PIN PEMENANG:454647LM lnfo Klik:www.kejutanbimatri.blogspot.com
Case Folding		 :  slmt no anda mendptkan hadiah unit mobil honda jazz dr promo tri care dgn pin pemenanglm lnfo klik
Word Nomalisasi		 :  selamat nomor anda mendapatkan hadiah unit mobil honda jazz dari promo tri care dengan pin pemenanglm lnfo klik
Stopword Removal	 :  selamat nomor hadiah unit mobil honda jazz promo tri care pin pemenanglm lnfo klik


## STEMMING

In [304]:
!pip -q install sastrawi

In [305]:
# merubah kata menjadi kata dasar
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [306]:
# membuat function untuk stemming bahasa indonesia
def stemming(text):
    text = stemmer.stem(text)
    return text

In [307]:
data_sample = data['Teks'].iloc[696]
case_folding = casefolding(data_sample)
stopword_removal = stopwordremoval(case_folding)
stemming_text = stemming(stopword_removal)

print('Data Sample\t\t : ', data_sample)
print('Case Folding\t\t : ', case_folding)
print('Stopword Removal\t : ', stopword_removal)
print('Stemming\t\t : ', stemming_text)

Data Sample		 :  Btw magicomnya yg sedang Gais, gaada yg gede
Case Folding		 :  btw magicomnya yg sedang gais gaada yg gede
Stopword Removal	 :  magicomnya yg gais gaada yg gede
Stemming		 :  magicomnya yg gais gaada yg gede


## TEST PREPROCESSING PIPELINE

In [308]:
# membuat fungsi untuk menggabungkan seluruh langkah text preprocessing
def text_preprocessing_process(text):
  text = casefolding(text)
  text = textnormalize(text)
  text = stopwordremoval(text)
  text = stemming(text)
  return text

In [309]:
%%time
data['clean_teks'] = data['Teks'].apply(text_preprocessing_process)

CPU times: user 6min 59s, sys: 1.49 s, total: 7min 1s
Wall time: 7min 18s


In [310]:
data

,Teks,label,clean_teks
0,[PROMO] Beli paket Flash mulai 1GB di MY TELKO...,2,promo beli paket flash gb my app extra kuota g...
1,2.5 GB/30 hari hanya Rp 35 Ribu Spesial buat A...,2,gb rupiah ribu spesial pilih aktif promo sd no...
2,"2016-07-08 11:47:11.Plg Yth, sisa kuota Flash ...",2,langgan hormat sisa kuota flash kb download my...
3,"2016-08-07 11:29:47.Plg Yth, sisa kuota Flash ...",2,langgan hormat sisa kuota flash kb download my...
4,4.5GB/30 hari hanya Rp 55 Ribu Spesial buat an...,2,gb rupiah ribu spesial pilih aktif buru skb
...,...,...,...
1138,"Yooo sama2, oke nanti aku umumin di grup kelas",0,yooo oke umumin grup kelas
1139,😁 sebelumnya ga ad nulis kerudung. Kirain warn...,0,nulis kerudung kirain warna jins
1140,Mba mau kirim 300 ya,0,mbak kirim ya
1141,nama1 beaok bwrangkat pagi...mau cas atay tra...,0,nama beaok bwrangkat pagimau cas atay tranfer


In [311]:
# simpan data yang sudah dipreprocessing ke dalam file csv
data.to_csv('/content/drive/MyDrive/Colab Notebooks/Tugas Akhir Praktisi Text Mining/clean_data.csv')

## FEATURE ENGINEERING

In [312]:
# Memisahkan kolom feature dan target

x = data['clean_teks']
y = data['label']

In [313]:
x

,clean_teks
0,promo beli paket flash gb my app extra kuota g...
1,gb rupiah ribu spesial pilih aktif promo sd no...
2,langgan hormat sisa kuota flash kb download my...
3,langgan hormat sisa kuota flash kb download my...
4,gb rupiah ribu spesial pilih aktif buru skb
...,...
1138,yooo oke umumin grup kelas
1139,nulis kerudung kirain warna jins
1140,mbak kirim ya
1141,nama beaok bwrangkat pagimau cas atay tranfer


In [314]:
y

,label
0,2
1,2
2,2
3,2
4,2
...,...
1138,0
1139,0
1140,0
1141,0


# FEATURE EXTRACTION (TF-IDF dan N-Gram)

In [315]:
# save model
import pickle

#TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

#Uningram
vec_TF_IDF = TfidfVectorizer(ngram_range=(1,1))
vec_TF_IDF.fit(x)

x_tf_idf = vec_TF_IDF.transform(x)

pickle.dump(vec_TF_IDF.vocabulary_, open('/content/drive/MyDrive/Colab Notebooks/Tugas Akhir Praktisi Text Mining/feature_tf-idf.sav', 'wb'))

In [316]:
# menampilkan vocabulary dari tf-idf
vec_TF_IDF.vocabulary_

{'promo': 2296,
 'beli': 323,
 'paket': 2089,
 'flash': 870,
 'gb': 942,
 'my': 1881,
 'app': 162,
 'extra': 841,
 'kuota': 1550,
 'lte': 1653,
 'telpon': 2878,
 'mnthr': 1832,
 'buru': 480,
 'cek': 521,
 'tselmemytsel': 3013,
 'sk': 2692,
 'rupiah': 2504,
 'ribu': 2453,
 'spesial': 2751,
 'pilih': 2176,
 'aktif': 66,
 'sd': 2558,
 'november': 1990,
 'langgan': 1578,
 'hormat': 1117,
 'sisa': 2685,
 'kb': 1411,
 'download': 752,
 'mytelkomsel': 1883,
 'apps': 167,
 'kuotabeli': 1551,
 'hubung': 1141,
 'skb': 2693,
 'ekstra': 804,
 'pulsa': 2333,
 'internet': 1221,
 'bulan': 466,
 'sjk': 2691,
 'augsept': 217,
 'detail': 665,
 'iring': 1243,
 'tarif': 2842,
 'panjang': 2102,
 'hits': 1106,
 'armada': 180,
 'curi': 600,
 'hati': 1070,
 'tekan': 2870,
 'okcall': 2042,
 'informasi': 1192,
 'eks': 801,
 'loh': 1640,
 'internetan': 1222,
 'pakai': 2087,
 'volume': 3129,
 'ultima': 3066,
 'mbhr': 1739,
 'harga': 1058,
 'tariflokasi': 2844,
 'tselmefl': 3011,
 'coboy': 568,
 'jr': 1330,
 'heba

In [317]:
# melihat jumlah feature
print(len(vec_TF_IDF.get_feature_names_out()))

3252


In [318]:
# melihat feature apa saja yang ada didalam corpus

print(vec_TF_IDF.get_feature_names_out())

['aa' 'aamiiiin' 'aamiin' ... 'zjt' 'zona' 'ztkm']


In [319]:
x1 = vec_TF_IDF.transform(x).toarray()
data_tabular_tf_idf = pd.DataFrame(x1, columns=vec_TF_IDF.get_feature_names_out())
data_tabular_tf_idf

,aa,aamiiiin,aamiin,ab,abadi,abai,abbee,abdul,acara,acaratks,...,yudisium,yuk,yuks,yuni,yunit,zalora,zarkasi,zjt,zona,ztkm
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1139,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1140,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1141,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [320]:
data_tabular_tf_idf.iloc[10:20,60:70]

,akang,akangteteh,akbar,akreditasi,akses,aksi,aktif,aktifasi,aktivasi,aktivitas
10,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
11,0.0,0.0,0.0,0.0,0.000000,0.0,0.149444,0.0,0.000000,0.0
12,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
13,0.0,0.0,0.0,0.0,0.262305,0.0,0.000000,0.0,0.000000,0.0
14,0.0,0.0,0.0,0.0,0.000000,0.0,0.244053,0.0,0.382416,0.0
15,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
16,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
17,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
18,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
19,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0


# FEATURE SELECTION

In [321]:
x_train = np.array(data_tabular_tf_idf)
y_train = np.array(y)

In [322]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import chi2

chi2_features = SelectKBest(chi2, k = 3000)
X_kbest_features = chi2_features.fit_transform(x_train, y_train)

# untuk reduce feature
print('Original Feature Number', x_train.shape[1])
print('Reduced Feature Number', X_kbest_features.shape[1])

Original Feature Number 3252
Reduced Feature Number 3000


In [323]:
Data = pd.DataFrame(chi2_features.scores_, columns=['Nilai'])
Data

,Nilai
0,0.835608
1,0.419698
2,1.558607
3,0.716455
4,0.800674
...,...
3247,1.180239
3248,0.503162
3249,0.716455
3250,2.918739


In [324]:
# menampilkan data feature beserta nilainya

feature = vec_TF_IDF.get_feature_names_out()
feature
Data['fitur'] = feature
Data

,Nilai,fitur
0,0.835608,aa
1,0.419698,aamiiiin
2,1.558607,aamiin
3,0.716455,ab
4,0.800674,abadi
...,...,...
3247,1.180239,zalora
3248,0.503162,zarkasi
3249,0.716455,zjt
3250,2.918739,zona


In [325]:
# mengurutkan nilai feature terbaik

Data.sort_values(by='Nilai', ascending=False)

,Nilai,fitur
2089,47.917892,paket
942,47.483452,gb
1031,46.429890,hadiah
1550,45.216656,kuota
2179,39.383773,pin
...,...,...
1521,0.044714,kopi
307,0.044468,bca
1695,0.031579,maksimal
531,0.013294,cepat


In [327]:
# menampilkan mask pada feature yang akan diseleksi
mask = chi2_features.get_support()
mask


array([ True,  True,  True, ...,  True,  True,  True])

In [335]:
# menampilkan fitur yang terpilih berdasarkan nilai mask dan nilai tertinggi yang sudah ditetapkan pada chi square
new_feature = []
for bool, f in zip(mask, feature):
  if bool:
    new_feature.append(f)
  selected_feature = new_feature
selected_feature

['a']

In [ ]:
#